# Нахождение с помощью характеристического уравнения

In [15]:
import numpy as np
from scipy.optimize import fsolve

def characteristic_method(A):
    n = A.shape[0]
    
    # Характеристическая функция
    def char_eq(lmbda):
        return np.linalg.det(A - lmbda * np.eye(n))
    
    # Находим приближенные корни характеристического уравнения
    eigvals_approx = np.linalg.eigvals(A)
    eigvals = [fsolve(char_eq, val)[0] for val in eigvals_approx]
    
    eigvecs = []
    for val in eigvals:
        # Решаем систему уравнений (A - lambda*I)v = 0
        _, _, V = np.linalg.svd(A - val * np.eye(n))
        eigvecs.append(V.T[:, -1])
    
    # Нормализация собственных векторов
    eigvecs = [vec/np.linalg.norm(vec) for vec in eigvecs]

    return eigvals, eigvecs

# Метод итераций

In [17]:

def power_iteration(A, max_iter=1000, tol=1e-6):
    x0 = np.random.rand(A.shape[0])
    
    norm_x0 = np.sqrt(np.sum(x0**2))
    x0 = x0 / norm_x0

    for _ in range(max_iter):
        y = np.dot(A, x0)
        lambda_ = np.dot(x0, y)
        
        norm_y = np.sqrt(np.sum(y**2))
        x1 = y / norm_y
        
        if np.linalg.norm(x1 - x0) < tol:
            break
        x0 = x1

    return lambda_, x1


# Метод вращения

In [13]:
def jacobi_rotation(A, max_iter=1000, tol=1e-6):
    n = A.shape[0]
    V = np.eye(n)
    
    for _ in range(max_iter):
        p, q = np.unravel_index(np.abs(np.triu(A, 1)).argmax(), A.shape)
        
        if abs(A[p, q]) < tol:
            break
        
        if A[p, p] == A[q, q]:
            phi = np.pi / 4 if A[p, q] > 0 else -np.pi / 4
        else:
            tau = (A[p, p] - A[q, q]) / (2 * A[p, q])
            t = np.sign(tau) / (abs(tau) + np.sqrt(1 + tau**2))
            phi = np.arctan(t) / 2
        
        c = np.cos(phi)
        s = np.sin(phi)
        
        G = np.eye(n)
        G[p, p], G[q, q], G[p, q], G[q, p] = c, c, -s, s
        
        A = np.dot(G.T, np.dot(A, G))
        V = np.dot(V, G)
        
    return np.diagonal(A), V


In [18]:
A = np.array([[5, 2, 1],
[2, 4, 1],
[1, 1, 3]])
# Тестирование метода характеристического уравнения
eigvals, eigvecs = characteristic_method(A)
print("Метод характеристического уравнения:")
print("Собственные значения:", eigvals)
print("Собственные векторы:", eigvecs)
print()
# Тестирование метода итераций
lambda_, eigvec = power_iteration(A)
print("Метод итераций:")
print("Максимальное по модулю собственное значение:", lambda_)
print("Соответствующий собственный вектор:", eigvec)
print()
# Тестирование метода вращения
eigvals, eigvecs = jacobi_rotation(A)
print("Метод вращения:")
print("Собственные значения:", eigvals)
print("Собственные векторы:", eigvecs)

Метод характеристического уравнения:
Собственные значения: [7.048917339522306, 2.6431041321077906, 2.307978528369904]
Собственные векторы: [array([-0.73697623, -0.59100905, -0.32798528]), array([ 0.59100905, -0.32798528, -0.73697623]), array([-0.32798528,  0.73697623, -0.59100905])]

Метод итераций:
Максимальное по модулю собственное значение: 7.048917339511715
Соответствующий собственный вектор: [0.73697585 0.59100934 0.32798559]

Метод вращения:
Собственные значения: [7.04891734 2.30797853 2.64310413]
Собственные векторы: [[ 0.73697636 -0.32798649 -0.59100821]
 [ 0.59100889  0.73697705  0.32798372]
 [ 0.32798526 -0.59100736  0.7369776 ]]
